<h2 style="color: #1EB382;font-weight: bold;">Redshift Integration</h2>

This notebooks guides through the ingestion of Redshift data in the Hopsworks feature store. To follow this notebook users should have an existing Redshift cluster, if not, they can follow the AWS [documentation](https://docs.aws.amazon.com/ses/latest/DeveloperGuide/event-publishing-redshift-cluster.html).

The data for this tutorial is available in CSV format [here]()
Users should create the following table in Redshift
```sql
CREATE TABLE telco(
    customer_id varchar(200),
    gender varchar(200),
    senior_citizen integer,
    partner varchar(200),
    dependents varchar(200),
    tenure integer,
    phone_service varchar(200),
    multiple_lines varchar(200),
    internet_service varchar(200),
    online_security varchar(200),
    online_backup varchar(200),
    device_protection varchar(200),
    tech_support varchar(200),
    streaming_tv varchar(200),
    streaming_movies varchar(200),
    contract varchar(200),
    paperless_billing varchar(200),
    payment_method varchar(200),
    monthly_charges double precision,
    total_charges varchar(200),
    churn varchar(200)
)
```

and populate the table using the copy command:
```sql
COPY telco
FROM 's3://bucket/telco_customer_churn.csv'
IAM_ROLE 'arn:aws:iam::xxxxxxxxx:role/role_name'
FORMAT as CSV
FILLRECORD
```

Once the data has been imported into Redshift, we can start ingesting it into the Hopsworks Feature Store. 

<h3 style="color: #1EB382;font-weight: bold;">Storage Connector</h3>

The first step to be able to ingest Redshift data in the feature store is to configure a storage connector. 
Hopsworks storage connectors are meant as a centralized place to store and manage configurations to read and write to external system. 

The Redshift connector requires you to specify the following properties. Most of them are available in the properties area of your cluster in the Redshift UI. 

- Cluster identifier: The name of the cluster

- Database driver: You can use the default JDBC Redshift Driver `com.amazon.redshift.jdbc42.Driver` (More on this later)

- Database endpoint: The endpoint for the database. Should be in the format of `[UUID].eu-west-1.redshift.amazonaws.com`

- Database name: The name of the database to query

- Database port: The port of the cluster. Defaults to 5349


There are two options to authenticate with the Redshift cluster. The first option is to configure username and password. The password is stored in the secret store and made available to all the member of the project.

Alternatively a IAM role can be configured. When configured the HSFS library will use the IAM role to acquire a temporary credential to authenticate the specified user.

With regards to the database driver, the library to interact with Redshift *is not* part of the standard Hopsworks distribution and needs to be provided by the user. The library can be downloaded here: https://docs.aws.amazon.com/redshift/latest/mgmt/configure-jdbc-connection.html#download-jdbc-driver

In [10]:
import hsfs
# Connect to the Hopsworks feature store
connection = hsfs.connection()
# Retrieve the metadata handle
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

<h3 style="color: #1EB382;font-weight: bold;">On-Demand Feature Group</h3>

To be able to read data from the Redshift table, an on-demand feature group needs to be defined.
In Hopsworks on-demand feature groups allow users to register features stored on external systems (Redshift, RDS, S3) with the Hopsworks feature store. 

Registering external features with the Hopsworks feature store has several advantages. The first one is *provenance*, from the Hopsworks Web UI users are able to track which features are stored on which external systems and how they are computed. 
Additionally HSFS (the Python/Scala library used to interact with the feature store) provides the same APIs for on-demand feature groups as for feature groups available within Hopsworks itself (Cached feature groups). Thus abstracting the complexity of configuring data ingestion.

An on-demand feature group can be defined as follow:

In [11]:
# Retrieve the storage connector defined before
redshift_conn = fs.get_storage_connector("telco_redshift_cluster")

In [3]:
telco_on_dmd = fs.create_on_demand_feature_group(name="telco_redshift",
                                                version=2,
                                                query="select * from telco",
                                                description="On-demand feature group for telecom customer data",
                                                storage_connector=redshift_conn,
                                                statistics_config=True)

In [5]:
telco_on_dmd.save()

<h3 style="color: #1EB382;font-weight: bold;">Feature Engineering</h3>

On-demand feature groups can be used directly as a source for creating training datasets. This is often the case if a company is migrating to Hopsworks and there are already feature engineering pipelines in production writing data to Redshift.

This flexibility provided by Hopsworks allows users to hit the ground running from day 1, without having to rewrite their pipelines to take advantage of the benefits the Hopsworks feature store provides.

In [13]:
telco_on_dmd.select(['customer_id', 'internet_service', 'phone_service', 'total_charges', 'churn']).show(5)

+-----------+----------------+-------------+-------------+-----+
|customer_id|internet_service|phone_service|total_charges|churn|
+-----------+----------------+-------------+-------------+-----+
| 7590-VHVEG|             DSL|           No|        29.85|   No|
| 5575-GNVDE|             DSL|          Yes|       1889.5|   No|
| 3668-QPYBK|             DSL|          Yes|       108.15|  Yes|
| 7795-CFOCW|             DSL|           No|      1840.75|   No|
| 9237-HQITU|     Fiber optic|          Yes|       151.65|  Yes|
+-----------+----------------+-------------+-------------+-----+
only showing top 5 rows

On-demand feature groups can also be joined with cached feature groups to create training datasets. This document (https://docs.hopsworks.ai/generated/query_vs_dataframe/) explains in details how the HSFS joining APIs work and how they can be used to create training datasets.

If, however, Redshift contains raw data that needs to be feature engineered, users can invoke the HSFS API and retrieve a Spark DataFrame backed by the Redshift table. 

In [32]:
spark_df = telco_on_dmd.read()

In [28]:
from pyspark.sql.types import DoubleType
from pyspark.sql import functions as F
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

In [48]:
categoricalColumns = ['gender','senior_citizen','partner','dependents','phone_service','multiple_lines',
                      'internet_service', 'online_security', 'online_backup', 'device_protection', 'tech_support',
                      'streaming_tv', 'streaming_movies', 'contract', 'paperless_billing', 'payment_method', 'churn']

spark_df = spark_df.withColumn("total_charges", F.col("total_charges").cast(DoubleType()))\
                   .fillna(0)

stages = [] # stages in our Pipeline
output_cols = [('customer_id', 'customer_id')]
for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    output_col = categoricalCol + "_Index"
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=output_col)
    stages += [stringIndexer]
    output_cols += [(categoricalCol, output_col)]

pipeline = Pipeline(stages=stages)
dataset = pipeline.fit(spark_df).transform(spark_df)
telco_fg_df = dataset.selectExpr(["{} as {}".format(col[1], col[0]) for col in output_cols])

In [51]:
telco_fg_df.show(5)

+-----------+------+--------------+-------+----------+-------------+--------------+----------------+---------------+-------------+-----------------+------------+------------+----------------+--------+-----------------+--------------+-----+
|customer_id|gender|senior_citizen|partner|dependents|phone_service|multiple_lines|internet_service|online_security|online_backup|device_protection|tech_support|streaming_tv|streaming_movies|contract|paperless_billing|payment_method|churn|
+-----------+------+--------------+-------+----------+-------------+--------------+----------------+---------------+-------------+-----------------+------------+------------+----------------+--------+-----------------+--------------+-----+
| 7590-VHVEG|   1.0|           0.0|    1.0|       0.0|          1.0|           2.0|             1.0|            0.0|          1.0|              0.0|         0.0|         0.0|             0.0|     0.0|              0.0|           0.0|  0.0|
| 5575-GNVDE|   0.0|           0.0|    0

<h3 style="color: #1EB382;font-weight: bold;">Derived Feature Group</h3>

Finally users can save the output of their transformations in the feature store as a derived cached feature group.

Storing feature groups as cached feature groups within Hopsworks provides several benefits. First it allows users to leverage HUDI for upserts and time travel capabilities. As new data is ingested, new commits are tracked by Hopsworks allowing users to see what has changed over time.
Similarly to on-demand feature groups, at each commit, statistics are computed and tracked in Hopsworks, allowing users to understand how the data changes.

Cached feature groups can also be made available online (`online_enabled=True`) thus enabling low latency requests to augment inference vectors.

In [53]:
telco_fg = fs.create_feature_group(name="telco_customer_features",
                                version=1,
                                description="Telecom customer features",
                                online_enabled=True,
                                time_travel_format="HUDI",
                                primary_key=["customer_id"],
                                statistics_config=True)

In [ ]:
telco_fg.save(telco_fg_df)